# Scraping mine data with `.apply`

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Doing this programmatically

## First, scraping a single page

### Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

### Searching for a mine

Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm) and use Selenium to search for `3503598`

- *TIP: You might need to use the Selenium code to scroll down to the right spot on the page. Or not!*
- *TIP: Use `.send_keys` to type into the box*
- *TIP: On pages that never change, you can usually just use XPath if you're feeling lazy*

In [13]:
driver = webdriver.Chrome()

In [14]:
driver.get("https://arlweb.msha.gov/drs/drshome.htm")

In [15]:
text_input = driver.find_element_by_name('MineId')

In [16]:
driver.execute_script("arguments[0].scrollIntoView(true)", text_input)

In [17]:
text_input.send_keys('3503598')

In [18]:
button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
button.click()


### Finding reports

On the "Report Selection Page" (where you should be after you search), use Selenium to...

- Change the **Beginning Date** to `1/1/1995`
- Select the report type of `Violations`
- Click **Get Report**

.

- *TIP: Remember, if someone isn't on the page Selenium can't click it!*

In [19]:
date_input = driver.find_element_by_name('BDate')

In [21]:
date_input.send_keys('1/1/1995')

In [22]:
violations_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
violations_button.click()

In [23]:
report_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
report_button.click()

### Saving reports

Save all of the rows of data on that page into a new dataframe. Each column is its own column, **and you also need to save the URL under the 'Standard' column.** Here, I even made you a blank dictionary:

```python
data = {}
data['violator'] = ''
data['contract_id'] = ''
data['citation_no'] = ''
data['case_no'] = ''
data['date_issues'] = ''
data['final_order_date'] = ''
data['section_of_act'] = ''
data['date_terminated'] = ''
data['citation'] = ''
data['s_and_s'] = ''
data['standard'] = ''
data['standard_url'] = ''
data['proposed_penalty'] = ''
data['citation_status'] = ''
data['current_penalty'] = ''
data['amount_paid'] = ''
```

- *TIP: Some of those table rows aren't what you want. How can you tell them apart from the good ones? (the previous mine owner ones are okay, I just mean the weird headers)*
- *TIP: I sense `.find_elements` + a lot of square brackets*
- *TIP: This is just like scraping a search results page!*
- *TIP: For the URL, you'll need to find the `a` inside of the cell*
- *TIP: class name is sadly not going to save your life here, because some of the `tr`s and `td`s have the same class! It's stupid. But there's a trick: CSS selectors! Something like `div#container` finds a `div` with the id of `container`, while `span.important` finds a `span` with the class of `important`. It should be helpful! And use `.find_elements_by_` + tab to see what the command is*

In [95]:
data_table1 = driver.find_elements_by_tag_name('tr')

for info in data_table1[36:]:
    print(info.text)

Newberg Rock & Dirt    8790964   000383840 4/13/2015 7/17/2015  104(a) 4/15/2015 C N 56.4101 100.00 Closed 100.00  100.00 
Newberg Rock & Dirt    8790962   000383840 4/10/2015 7/17/2015  104(a) 4/10/2015 C N 56.14107(a) 100.00 Closed 100.00  100.00 
Newberg Rock & Dirt    8790963   000383840 4/10/2015 7/17/2015  104(a) 4/13/2015 C Y 56.15020 873.00 Closed 873.00  873.00 
Newberg Rock & Dirt    8694458   000345066 1/29/2014 4/12/2014  104(a) 1/29/2014 C N 56.12018 100.00 Closed 100.00  100.00 
Newberg Rock & Dirt    8694457   000345066 1/29/2014 4/12/2014  104(a) 1/29/2014 C N 56.12004 100.00 Closed 100.00  100.00 
Newberg Rock & Dirt    8694456   000345066 1/28/2014 4/12/2014  104(a) 1/29/2014 C N 56.14132(b)(1) 100.00 Closed 100.00  100.00 
Newberg Rock & Dirt    8694454   000345066 1/28/2014 4/12/2014  104(a) 1/29/2014 C Y 56.12032 243.00 Closed 243.00  243.00 
Newberg Rock & Dirt    8694455   000345066 1/28/2014 4/12/2014  104(a) 1/29/2014 C N 56.12004 100.00 Closed 100.00  100.00 


In [101]:
data_table2 = data_table1[36].find_elements_by_tag_name('td')

for info in data_table2:
    print(info.text)

Newberg Rock & Dirt
  
8790964  
000383840
4/13/2015
7/17/2015 
104(a)
4/15/2015
C
N
56.4101
100.00
Closed
100.00 
100.00 


In [71]:
data_table2[0]

'Newberg Rock & Dirt'

# Join everything in one cell

In [1]:
data = {}

data_table1 = driver.find_elements_by_tag_name('tr')

for info in data_table1[36:]:
    data_table2 = info.find_elements_by_tag_name('td')

data['violator'] = data_table2[0].text
data['contract_id'] = data_table2[1].text
data['citation_no'] = data_table2[2].text
data['case_no'] = data_table2[3].text
data['date_issues'] = data_table2[4].text
data['final_order_date'] = data_table2[5].text
data['section_of_act'] = data_table2[6].text
data['date_terminated'] = data_table2[7].text
data['citation'] = data_table2[8].text
data['s_and_s'] = data_table2[9].text
data['standard'] = data_table2[10].text
data['standard_url'] = data_table2[10].find_element_by_xpath('//*[@id="anch_66"]').get_attribute('href')  
data['proposed_penalty'] = data_table2[11].text
data['citation_status'] = data_table2[12].text
data['current_penalty'] = data_table2[13].text
data['amount_paid'] = data_table2[14].text


data

NameError: name 'driver' is not defined

### Saving that data

Save the dataframe to a CSV file called `3503598-violations.csv` (that's the TDLR code)

## Put that all in ONE cell that runs correctly

The **entire process**, from searching to saving as a CSV

# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them? If so, make them match.

- *TIP: You can zero fill if you want, but another option is that when reading in a CSV, `dtype='str'` will force everything to be a string*

### Convert your one-cell scraper into a function, and use it on each row of our dataset

- *TIP: You'll be using `.apply`*
- *TIP: You won't be joining this back into your dataframe, so you don't need to `return` anything or `join` or any of that.*
- *TIP: Be careful of your **other variable names** - if you're calling the thing you're sending your function `row`, you can't use it anywhere else (like in your loop)*
- *TIP: **BE CAREFUL WHAT YOU NAME YOUR DATAFRAMES.** If you name the citations dataframe `df` it can overwrite your mine ID `df`*
- *TIP: You'll be saving a dataframe each time*
- *TIP: Be sure you change everything that refers to the mine ID to refer to the current row's ID instead of `3503598`*
- *TIP: BE SURE TO CHANGE EVERYTHING THAT REFERS TO THE MINE ID*
- *TIP: EVERYTHING, EVERYTHING, EVERYTHING! Look at the end of your function! Maybe I'm overreacting, I don't know.*
- *TIP: If you hit an error about list index out of range, see what line it's happening on and go look at the page. What's different about this page than the previous ones? (answer: the last three columns!) If you assign those columns later using `try`/`except` you should be able to get some data from those rows without throwing it all out. If you can't figure it out, just wrap it all in try/except and give up on those rows*
- *TIP: Some of the standards might not have links, either, so you might want to wrap that in a `try`/`except`, too!*

# Okay, now do it for ALL of the mines

Open up `mines.csv` using pandas and do the same thing, it will just be for more mines this time.